# Project 3A: Depression and Loneliness Text Analysis

## Problem Statement

With the increasing comparison behaviour due to the constant flow of information from the internet and social media, coupled with the lack of physical social interactions, there is an increasing trend in both depression and loneliness among younger millennials [[1]](https://www.healthline.com/health-news/millennial-depression-on-the-rise#Is-social-media-to-blame?). As a data scientist working in a mental healthcare institutition, we are interested to first explore the top 10 differences in word expression between depression and loneliness behavioural through the social media forum. We will be using 4 types of models namely the null model, multinomial naive bayes, k-nearest neighbours and logistic regression to predict the identification of depression and loneliness of the social media forum. Of these models, we will select the model that are able to provide the highest accuracy with more than 80% that takes into the account of the similarity between depression and loneliness. This will allows us to gain an insight to understand the early behavioural symptoms through their words usage and instill this as part of the assessment and identification when communicating with person in need and provide better assistance through other communication means including the chatbox and the helpline with the model built with more than 80% accuracy.

### Overall Contents:
- [Background](#1.-Background) **(In this notebook)**
- [Data Collection](#2.-Data-Collection) **(In this notebook)**
- Data Cleaning
- Exploratory Data Analysis
- Preprocessing
- Modeling
- Evaluation
- Conclusions and Recommendations

## 1. Background

Depression and loneliness are terms that are often associated together due to the common belief that loneliness is a subset of depression and they have high similarity such as the symptoms of being in pain and helplessness [[2]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4225959/). 
However, the main distinction between depression and loneliness is loneliness is a transient emotional state that corresponds to a person needs for belonging and connection while depression is a mental health condition that can significantly affect a person's ability to function normally with its persistent negative feelings of emptiness, worthlessness, hopelessness and sadness [[3]](https://www.healthline.com/health/loneliness-and-depression) [[4]](https://www.mentalhelp.net/depression/hotline/). Loneliness condition can be improved should those needs be met, while depression on the other hand, requires treatment and can lead to suicidal on severe cases [[5]](https://salienceneuro.com/long-term-effects-of-depression-2/). Nevertheless, a person experiencing loneliness should not be left unattended as it is an indicator of social well-being and prolonged loneliness can also lead to other mental health conditions including depression [[3]](https://www.healthline.com/health/loneliness-and-depression). Hence, it is important to distinguish the differences of loneliness and depression to allow for early detection and offer the appropriate treatment to combat this before it prolonged to undesire consequences. 

Globally, depression is ranked as the single largest contributor to global disability by World Health Organization (WHO) whereby, it is estimated to exceed 300 million people (4.4% of the world's population) having depression in 2015 and it is a major contributor to suicide deaths (up to 800 thosands per year) [[6]](https://www.who.int/publications/i/item/depression-global-health-estimates). The high prevalence rate of depression and suicidal is due to barrier to effective care including stigma surrounding mental disorders, lack of awareness and inaccurate assessment  [[7]](https://www.who.int/news-room/fact-sheets/detail/depression). With the increase usage of internet and social media, there is an increasing trend in both depression and loneliness among younger millenials in which the usage of these platforms encourage unintentional comparison behaviour with their peers and instill a sense of disconnect from real-life social interaction[[8]](https://pdxscholar.library.pdx.edu/cgi/viewcontent.cgi?article=6646&context=open_access_etds). This leave them feeling insecure and unaccomplished, thus resulting in lower self-esteem, anxiety and depression. Thus, it is important to create awareness and reachout to them by being available and accessible for support or assistance should they experience depression or loneliness, which is a factor leading to depression. 

Hence, as a data scientist working in a mental healthcare institutition, I will be handling on this project and our goals are the following:
1. To identify the top 10 differences between depression and loneliness behavioural through the word expression in social media forum

2. To build models such as null model, multinomial naive bayes, k-nearest neighbours and logistic regression to predict the identification of depression and loneliness behavioural with using the social media forum as a platform to measure the accuracy of the model prediction in this identification.

3. To select a model among the models built with the highest accuracy and having more than 80% accuracy to predict the identification of depression and loneliness behavioural in the social media forum, with taking an account of the similarity of depression and loneliness. A recent study suggests that 18% of the loneliness is responsible for depression in older adults. [[9]](https://www.sciencedaily.com/releases/2020/11/201109184947.htm). As the quantitative measurement of loneliness responsible for depression in young adults are not commonly studied, we will use 20% as a benchmark with reference to the study.

By knowing top 10 words that differences that distinguish the depression and loneliness behaviourial, the mental institution will be able to use this insight to instill these words as part of their assessment or identification and offer help when communicating with person in need. In addition, the model built with more than 80% accuracy will enable to provide a better assistance through other communication means including chatbox and the helpline.

### 1.1 Datasets

The dataset contains the posts that were published under the social news website, [reddit](https://www.reddit.com) between September 2020 to March 2021. 

The dataset that are to be collected are as followed:-

* depression_data (5000 posts)
* lonely_data (5000 posts)

The data source below are obtained from database [reddit](https://www.reddit.com):
* [depression](https://www.reddit.com/search/?q=depression) 
* [foreveralone](https://www.reddit.com/search?q=foreveralone&type=link)

## 2. Data Collection

### 2.1 Libraries Import

In [1]:
# Imports:
import requests
import time
import pandas as pd
import numpy as np

### 2.2 Data Collection

**Overview**

We will be collecting about 5000 posts from the forum of depression and lonely in reddit website prior to 1st April 2021 (using 1st April 2021 as the starting time). The criteria set for each collection is that there must be contents in the description of the posts labeled as "selftext" and there are no videos. For each forum, we will collect 1000 posts from the website using api.pushshift.io (https://api.pushshift.io/reddit/) for each collection and the remaining posts to top up to 5000 posts. After the collection, we will compile the posts into their respective dataframe.

Note: 
1. All codes will be placed under comment to prevent re-collection of data from reddit
2. In this project, foreveralone and loneliness or lonely are used interchangeable.

### 2.2.1 Define function for data collection from reddit

In [2]:
# Define the function for data collection from reddit
def collection_add_reddit(number_posts, url, subreddit, size, before):
    """To collect the posts from subreddit in reddit by specifying the created_utc to collect 
    the posts before this time and the size is the numbers of posts per pull
    The output is a dataframe containing the posts"""
    url = url 
    dataframe = pd.DataFrame()
    count = 0
    
    while len(dataframe) < number_posts: # run the code below till it reaches more than the number of post requested
        if len(dataframe) == 0: # for the first data collection
            subreddit = subreddit # the forum to collect
            size = size # number of posts of collection
            before = before # to collect the posts prior to stated time (utc time format)
            params = {"subreddit" : subreddit,'size' : size, "before": before}
            res = requests.get(url,params)
            print(f"for request {count}, status code is {res.status_code}")
            data = res.json()
            #removal of selftext having no contents and where there is video
            new_data = pd.DataFrame(data['data'])
            new_data = new_data[new_data["selftext"]!='[removed]']
            new_data = new_data[new_data["selftext"]!='[deleted]']
            new_data = new_data[new_data["selftext"]!= ""]
            new_data = new_data[new_data["is_video"]!= True]
            new_data = new_data.reset_index(drop = True)
            dataframe = pd.concat([dataframe, new_data]) #combine the new dataframe with existing dataframe
            dataframe = dataframe.reset_index(drop = True)
            time.sleep(np.random.randint(10,30)) #Time interval before next collection
            print(f"for request {count}, total rows are {dataframe.shape[0]}")
            count += 1
        else: # subsequent data collection
            subreddit = subreddit # the forum to collect
            size = size # number of posts of collection
            number = len(dataframe) - 1 # to call out the last row
            utc_before = dataframe.created_utc[number] # to collect the posts prior to the earliest created time of the dataframe
            params = {"subreddit" : subreddit,'size' : size,"before": utc_before}
            res = requests.get(url,params)
            print(f"for request {count}, status code is {res.status_code}")
            data = res.json()
            #removal of selftext having no contents and where there is video
            new_data = pd.DataFrame(data['data'])
            new_data = new_data[new_data["selftext"]!='[removed]']
            new_data = new_data[new_data["selftext"]!='[deleted]']
            new_data = new_data[new_data["selftext"]!= ""]
            new_data = new_data[new_data["is_video"]!= True]
            new_data = new_data.reset_index(drop = True)
            dataframe = pd.concat([dataframe, new_data]) #combine the new dataframe with existing dataframe
            dataframe = dataframe.reset_index(drop = True)
            time.sleep(np.random.randint(10,30)) #Time interval before next collection
            print(f"for request {count}, total rows are {dataframe.shape[0]}")
            count += 1
    return dataframe

### 2.2.2 Collection from reddit: First 1000 post

**Depression**

In [3]:
#depression_d1 = collection_add_reddit(1000,'https://api.pushshift.io/reddit/search/submission', 'depression', 100, 1617206400)

for request 0, status code is 200
for request 0, total rows are 80
for request 1, status code is 200
for request 1, total rows are 158
for request 2, status code is 200
for request 2, total rows are 245
for request 3, status code is 200
for request 3, total rows are 331
for request 4, status code is 200
for request 4, total rows are 411
for request 5, status code is 200
for request 5, total rows are 500
for request 6, status code is 200
for request 6, total rows are 582
for request 7, status code is 200
for request 7, total rows are 672
for request 8, status code is 200
for request 8, total rows are 758
for request 9, status code is 200
for request 9, total rows are 839
for request 10, status code is 200
for request 10, total rows are 920
for request 11, status code is 200
for request 11, total rows are 995
for request 12, status code is 200
for request 12, total rows are 1079


In [4]:
# To check the data collected from depression forum
#depression_d1.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,author_premium,...,url,whitelist_status,wls,post_hint,preview,removed_by_category,author_cakeday,author_flair_background_color,author_flair_text_color,banned_by
0,[],False,hatetheseparties521,None,[],None,text,t2_5h53y7nh,False,False,...,https://www.reddit.com/r/depression/comments/m...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,jaksiejsndnd,None,[],None,text,t2_b8zl7s08,False,False,...,https://www.reddit.com/r/depression/comments/m...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,PIRY_TV,None,[],None,text,t2_t5i70,False,False,...,https://www.reddit.com/r/depression/comments/m...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,inkv0ids,None,[],None,text,t2_9hpwrqhh,False,False,...,https://www.reddit.com/r/depression/comments/m...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,kamakirimen,None,[],None,text,t2_5xctywkc,False,False,...,https://www.reddit.com/r/depression/comments/m...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# To check the columns in the dataframe
#depression_d1.keys()

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_patreon_flair',
       'author_premium', 'awarders', 'can_mod_post', 'contest_mode',
       'created_utc', 'domain', 'full_link', 'gildings', 'id',
       'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_richtext',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler',
       'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
       'subreddit_type', 'suggested_sort', 'thumbnail', 'title',
       'total_awards_received', 'treatment_tags

In [6]:
# To check the utc time the post is created for the last post
#depression_d1.created_utc[1074]

1617000573

**Lonely**

In [8]:
#lonely_d1 = collection_add_reddit(1000,'https://api.pushshift.io/reddit/search/submission', 'foreveralone', 100, 1617206400)

for request 0, status code is 200
for request 0, total rows are 45
for request 1, status code is 200
for request 1, total rows are 97
for request 2, status code is 200
for request 2, total rows are 149
for request 3, status code is 200
for request 3, total rows are 209
for request 4, status code is 200
for request 4, total rows are 262
for request 5, status code is 200
for request 5, total rows are 318
for request 6, status code is 200
for request 6, total rows are 363
for request 7, status code is 200
for request 7, total rows are 409
for request 8, status code is 200
for request 8, total rows are 460
for request 9, status code is 200
for request 9, total rows are 510
for request 10, status code is 200
for request 10, total rows are 557
for request 11, status code is 200
for request 11, total rows are 608
for request 12, status code is 200
for request 12, total rows are 671
for request 13, status code is 200
for request 13, total rows are 727
for request 14, status code is 200
for req

### 2.2.3 Collection from reddit: Second 1000 post

**Depression**

In [9]:
#depression_d2 = collection_add_reddit(1000,'https://api.pushshift.io/reddit/search/submission', 'depression', 100, depression_d1.created_utc[(len(depression_d1) - 1)])

for request 0, status code is 200
for request 0, total rows are 84
for request 1, status code is 200
for request 1, total rows are 169
for request 2, status code is 200
for request 2, total rows are 247
for request 3, status code is 200
for request 3, total rows are 327
for request 4, status code is 200
for request 4, total rows are 408
for request 5, status code is 200
for request 5, total rows are 493
for request 6, status code is 200
for request 6, total rows are 573
for request 7, status code is 200
for request 7, total rows are 653
for request 8, status code is 200
for request 8, total rows are 732
for request 9, status code is 200
for request 9, total rows are 817
for request 10, status code is 200
for request 10, total rows are 902
for request 11, status code is 200
for request 11, total rows are 980
for request 12, status code is 200
for request 12, total rows are 1064


**Lonely**

In [10]:
#lonely_d2 = collection_add_reddit(1000,'https://api.pushshift.io/reddit/search/submission', 'foreveralone', 100, lonely_d1.created_utc[(len(lonely_d1) - 1)])

for request 0, status code is 200
for request 0, total rows are 48
for request 1, status code is 200
for request 1, total rows are 106
for request 2, status code is 200
for request 2, total rows are 169
for request 3, status code is 200
for request 3, total rows are 223
for request 4, status code is 200
for request 4, total rows are 293
for request 5, status code is 200
for request 5, total rows are 340
for request 6, status code is 200
for request 6, total rows are 390
for request 7, status code is 200
for request 7, total rows are 441
for request 8, status code is 200
for request 8, total rows are 505
for request 9, status code is 200
for request 9, total rows are 555
for request 10, status code is 200
for request 10, total rows are 615
for request 11, status code is 200
for request 11, total rows are 669
for request 12, status code is 200
for request 12, total rows are 728
for request 13, status code is 200
for request 13, total rows are 788
for request 14, status code is 200
for re

### 2.2.4 Collection from reddit: Third 1000 post

**Depression**

In [11]:
#depression_d3 = collection_add_reddit(1000,'https://api.pushshift.io/reddit/search/submission', 'depression', 100, depression_d2.created_utc[(len(depression_d2) - 1)])

for request 0, status code is 200
for request 0, total rows are 84
for request 1, status code is 200
for request 1, total rows are 169
for request 2, status code is 200
for request 2, total rows are 255
for request 3, status code is 200
for request 3, total rows are 342
for request 4, status code is 200
for request 4, total rows are 428
for request 5, status code is 200
for request 5, total rows are 518
for request 6, status code is 200
for request 6, total rows are 608
for request 7, status code is 200
for request 7, total rows are 690
for request 8, status code is 200
for request 8, total rows are 775
for request 9, status code is 200
for request 9, total rows are 863
for request 10, status code is 200
for request 10, total rows are 952
for request 11, status code is 200
for request 11, total rows are 1037


**Lonely**

In [12]:
#lonely_d3 = collection_add_reddit(1000,'https://api.pushshift.io/reddit/search/submission', 'foreveralone', 100, lonely_d2.created_utc[(len(lonely_d2) - 1)])

for request 0, status code is 200
for request 0, total rows are 32
for request 1, status code is 200
for request 1, total rows are 70
for request 2, status code is 200
for request 2, total rows are 98
for request 3, status code is 200
for request 3, total rows are 136
for request 4, status code is 200
for request 4, total rows are 171
for request 5, status code is 200
for request 5, total rows are 197
for request 6, status code is 200
for request 6, total rows are 227
for request 7, status code is 200
for request 7, total rows are 251
for request 8, status code is 200
for request 8, total rows are 292
for request 9, status code is 200
for request 9, total rows are 327
for request 10, status code is 200
for request 10, total rows are 357
for request 11, status code is 200
for request 11, total rows are 393
for request 12, status code is 200
for request 12, total rows are 428
for request 13, status code is 200
for request 13, total rows are 461
for request 14, status code is 200
for requ

### 2.2.5 Collection from reddit: Fourth 1000 post

**Depression**

In [13]:
#depression_d4 = collection_add_reddit(1000,'https://api.pushshift.io/reddit/search/submission', 'depression', 100, depression_d3.created_utc[(len(depression_d3) - 1)])

for request 0, status code is 200
for request 0, total rows are 82
for request 1, status code is 200
for request 1, total rows are 160
for request 2, status code is 200
for request 2, total rows are 244
for request 3, status code is 200
for request 3, total rows are 327
for request 4, status code is 200
for request 4, total rows are 415
for request 5, status code is 200
for request 5, total rows are 482
for request 6, status code is 200
for request 6, total rows are 563
for request 7, status code is 200
for request 7, total rows are 645
for request 8, status code is 200
for request 8, total rows are 727
for request 9, status code is 200
for request 9, total rows are 811
for request 10, status code is 200
for request 10, total rows are 894
for request 11, status code is 200
for request 11, total rows are 977
for request 12, status code is 200
for request 12, total rows are 1064


**Lonely**

In [14]:
#lonely_d4 = collection_add_reddit(1000,'https://api.pushshift.io/reddit/search/submission', 'foreveralone', 100, lonely_d3.created_utc[(len(lonely_d3) - 1)])

for request 0, status code is 200
for request 0, total rows are 61
for request 1, status code is 200
for request 1, total rows are 124
for request 2, status code is 200
for request 2, total rows are 183
for request 3, status code is 200
for request 3, total rows are 237
for request 4, status code is 200
for request 4, total rows are 291
for request 5, status code is 200
for request 5, total rows are 341
for request 6, status code is 200
for request 6, total rows are 400
for request 7, status code is 200
for request 7, total rows are 453
for request 8, status code is 200
for request 8, total rows are 519
for request 9, status code is 200
for request 9, total rows are 573
for request 10, status code is 200
for request 10, total rows are 630
for request 11, status code is 200
for request 11, total rows are 683
for request 12, status code is 200
for request 12, total rows are 741
for request 13, status code is 200
for request 13, total rows are 801
for request 14, status code is 200
for re

### 2.2.6 Collection from reddit: Remaining post to 5000 posts

**Depression**

In [15]:
#Calculate the remaining posts
# dep_rem_post = 5000 - len(depression_d1) - len(depression_d2) - len(depression_d3) - len(depression_d4)
# dep_rem_post

756

In [16]:
# Collect the remaining 700 posts
#depression_d5 = collection_add_reddit(750,'https://api.pushshift.io/reddit/search/submission', 'depression', 100, depression_d4.created_utc[(len(depression_d4) - 1)])

for request 0, status code is 200
for request 0, total rows are 85
for request 1, status code is 200
for request 1, total rows are 169
for request 2, status code is 200
for request 2, total rows are 254
for request 3, status code is 200
for request 3, total rows are 338
for request 4, status code is 200
for request 4, total rows are 419
for request 5, status code is 200
for request 5, total rows are 497
for request 6, status code is 200
for request 6, total rows are 575
for request 7, status code is 200
for request 7, total rows are 651
for request 8, status code is 200
for request 8, total rows are 738
for request 9, status code is 200
for request 9, total rows are 824


**Lonely**

In [17]:
#Calculate the remaining posts
# lon_rem_post = 5000 - len(lonely_d1) - len(lonely_d2) - len(lonely_d3) - len(lonely_d4)
# lon_rem_post

892

In [18]:
# Collect the remaining 900 posts
#lonely_d5 = collection_add_reddit(900,'https://api.pushshift.io/reddit/search/submission', 'foreveralone', 100, lonely_d4.created_utc[(len(lonely_d4) - 1)])

for request 0, status code is 200
for request 0, total rows are 63
for request 1, status code is 200
for request 1, total rows are 107
for request 2, status code is 200
for request 2, total rows are 161
for request 3, status code is 200
for request 3, total rows are 229
for request 4, status code is 200
for request 4, total rows are 290
for request 5, status code is 200
for request 5, total rows are 348
for request 6, status code is 200
for request 6, total rows are 406
for request 7, status code is 200
for request 7, total rows are 449
for request 8, status code is 200
for request 8, total rows are 500
for request 9, status code is 200
for request 9, total rows are 564
for request 10, status code is 200
for request 10, total rows are 626
for request 11, status code is 200
for request 11, total rows are 690
for request 12, status code is 200
for request 12, total rows are 750
for request 13, status code is 200
for request 13, total rows are 800
for request 14, status code is 200
for re

### 2.2.7 Compilation of posts

**Depression**

In [30]:
# depression_d1_d2 = pd.concat([depression_d1, depression_d2])
# depression_d1_d3 = pd.concat([depression_d1_d2, depression_d3])
# depression_d1_d4 = pd.concat([depression_d1_d3, depression_d4])
# depression_data = pd.concat([depression_d1_d4, depression_d5])
# depression_data = depression_data.reset_index(drop = True)
# print(f"The depression post time frame are from {depression_data.created_utc[(len(depression_data)-1)]} to {depression_data.created_utc[0]}")
# print(f"The total depression_data shape is {depression_data.shape}")

The depression post time frame are from 1615420610 to 1617206312
The total depression_data shape is (5068, 66)


**Lonely**

In [31]:
# lonely_d1_d2 = pd.concat([lonely_d1, lonely_d2])
# lonely_d1_d3 = pd.concat([lonely_d1_d2, lonely_d3])
# lonely_d1_d4 = pd.concat([lonely_d1_d3, lonely_d4])
# lonely_data = pd.concat([lonely_d1_d4, lonely_d5])
# lonely_data = lonely_data.reset_index(drop = True)
# print(f"The lonely post time frame are from {lonely_data.created_utc[(len(lonely_data)-1)]} to {lonely_data.created_utc[0]}")
# print(f"The lonely_data shape is {lonely_data.shape}")

The lonely post time frame are from 1600057348 to 1617198755
The lonely_data shape is (5026, 80)


### 2.2.7 To verify the removal of selftext without contents or videos

**Depression**

In [32]:
#depression_data[depression_data["selftext"] =='[removed]'].count()[['selftext']]

selftext    0
dtype: int64

In [33]:
#depression_data[depression_data["selftext"] =='[deleted]'].count()[['selftext']]

selftext    0
dtype: int64

In [34]:
#depression_data[depression_data["selftext"] == ""].count()[['selftext']]

selftext    0
dtype: int64

In [35]:
#depression_data[depression_data["is_video"] == True].count()[['selftext']]

selftext    0
dtype: int64

**Lonely**

In [36]:
#lonely_data[lonely_data["selftext"] =='[removed]'].count()[['selftext']]

selftext    0
dtype: int64

In [37]:
#lonely_data[lonely_data["selftext"] =='[deleted]'].count()[['selftext']]

selftext    0
dtype: int64

In [38]:
#lonely_data[lonely_data["selftext"] == ""].count()[['selftext']]

selftext    0
dtype: int64

In [39]:
#lonely_data[lonely_data["is_video"] == True].count()[['selftext']]

selftext    0
dtype: int64

## 2.3. Summary

* About five thousands posts from depression and lonely forum have successfully collected from the reddit website.
* The posts were compiled to depression_data and lonely_data respectively.
* These posts were be exported as a csv file and be cleaned in the next section.

## Exporting Data

In [40]:
# Placed the # to refrain from executing
#depression_data.to_csv("../data/depression_data.csv", index = False)
#lonely_data.to_csv("../data/lonely_data.csv", index = False) 

## References

[1] C. Curley, "Why Millennial Depression Is on the Rise," *Healthline Media*, March 11, 2019. [Online]. Available: https://www.healthline.com/health-news/millennial-depression-on-the-rise#Is-social-media-to-blame? [Accessed: Apr. 17, 2021].

[2] R. Mushtaq, S. Shoib, T. Shah, and S. Mushtaq, "Relationship Between Loneliness, Psychiatric Disorders and Physical Health ? A Review on the Psychological Aspects of Loneliness," *Journal of Clinical and Diagnostic Research,* vol.8, no.9, WE01-04, September 20, 2014. doi: 10.7860/JCDR/2014/10077.4828 [Online]. Available: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4225959/ [Accessed: Apr. 17, 2021].

[3] C. Raypole, "Loneliness and Depression: What's the Connection?," *Healthline Media*, July 2, 2020. [Online]. Available: https://www.healthline.com/health/loneliness-and-depression [Accessed: Apr. 17, 2021].

[4] "Depression Hotline Number," *MentalHelp.net*. [Online]. Available: https://www.mentalhelp.net/depression/hotline/ [Accessed: Apr. 17, 2021].

[5] "The Long-term Effects of Not Seeking Treatment for Depression," *Salience TMS Neuro Solutions*, March 23, 2020. [Online]. Available: https://salienceneuro.com/long-term-effects-of-depression-2/ [Accessed: Apr. 17, 2021].

[6] "Depression and Other Common Mental Disorders," *World Health Organization*, January 3, 2017. [Online]. Available: https://www.who.int/publications/i/item/depression-global-health-estimates [Accessed: Apr. 17, 2021].

[7] "Depression," *World Health Organization*, January 30, 2020. [Online]. Available: https://www.who.int/news-room/fact-sheets/detail/depression [Accessed: Apr. 17, 2021].

[8] J. E. Johnson, "Social Media Use, Social Comparison, and Loneliness," *Dissertations and
Theses*, Paper 5571, 2020, pp. 1-35. doi: 10.15760/etd.7445 [Online]. Available: https://pdxscholar.library.pdx.edu/cgi/viewcontent.cgi?article=6646&context=open_access_etds [Accessed: Apr. 17, 2021].

[9] "Loneliness a leading cause of depression in older adults," *ScienceDaily*, 2021. [Online]. Available: https://www.sciencedaily.com/releases/2020/11/201109184947.htm [Accessed: Apr. 22, 2021].
